In [15]:
import numpy as np
import pandas as pd
import h5py
from scipy import signal
import nibabel as nib

import sys
sys.path.insert(0, "../")
from config import *
from utils import load_nifti
import copy

In [2]:
import matplotlib.pyplot as plt

In [3]:
doc_dir

'/analysis/ritter/data/MS/Test/file_list_HC_MS_BET_FLAIR.csv'

In [4]:
!ls /analysis/ritter/data/MS/Test/file_list_HC_MS_BET_FLAIR.csv

/analysis/ritter/data/MS/Test/file_list_HC_MS_BET_FLAIR.csv


In [5]:
df = pd.read_csv(doc_dir)

In [6]:
cross_kernel = np.zeros(shape=(3, 3, 3))
cross_kernel[1,:,:] = 1.
cross_kernel[:,1,:] = 1.
cross_kernel[:,:,1] = 1.

In [42]:
def fill_lesions(dataset, pad_lesions=True):
    for idx, row in dataset.iterrows():
        path = row["path"]
        print(path)
        # FLAIR image path
        path = path.replace("/analysis/share/Ritter/MS", "/analysis/ritter/data/MS")
        # lesion mask path
        lm_path = path.replace("BET_", "")
        lm_path = lm_path.replace("FLAIR", "T2LESION_QC")
        # segmentation files path
        wm_path = path.replace("BET_", "c2")
        wm_path = wm_path.replace("FLAIR.nii.gz", "MPRAGE.nii")
        gm_path = wm_path.replace("c2", "c1")
        csf_path = wm_path.replace("c2", "c3")
        
        # in case of no lesion mask fill with random values
        try:
            scan, nifti = load_nifti(path, incl_header=True)
            lm = load_nifti(lm_path)
            if np.max(lm) > 1 or np.min(lm) < 0:
                print("Outside of normal range. Max {} Min {}".format(np.max(lm), np.min(lm)))
                lm = np.clip(lm, 0, 1)
            #if np.mean(lm) == 1:
            #    print("Lesion mean equal to 1. Change to 0 instead.")
            #    lm[:] = 0
            # pad lesion masks
            if pad_lesions:
                if np.max(lm) > 0:
                    pad_lm = signal.convolve(lm, cross_kernel, mode="same")
                    pad_lm_bin = np.zeros_like(pad_lm)
                    pad_lm_bin[pad_lm>=np.max(pad_lm)/8] = 1.
                    lm = pad_lm_bin
                #plt.figure(figsize=(10, 5))
                #plt.subplot(1, 2, 1)
                #plt.imshow(lm[:,:,96], cmap='gray')
                #plt.subplot(1, 2, 2)
                #plt.imshow(pad_lm_bin[:,:,96], cmap='gray')
                #plt.show()
                
            
            # compute white matter mask
            wm = load_nifti(wm_path)
            gm = load_nifti(gm_path)
            csf = load_nifti(csf_path)
            wm_mask = np.zeros_like(wm)
            wm_mask[np.greater(wm, gm) & np.greater(wm,csf)] = 1.
            
            #plt.imshow(wm_mask[:,:,96], cmap='gray')
            #plt.show()
            
            # extract normal appearing white matter (NAWM)
            nawm = wm_mask * scan
            #plt.imshow(nawm[:,:,96], cmap='gray')
            #plt.show()
            
            # NAWM distribution to sample from
            # np.random.normal(loc=mean, scale=std/2)
            
            """plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(scan[:,:,96], cmap='gray')
            plt.title("Before")
            """
            for sl in range(np.shape(scan)[2]):
                if lm[:,:,sl].any() > 0 and nawm[:,:,sl].any() > 0:
                    current_slice = np.copy(scan[:,:,sl])
                    lm_slice = np.copy(lm[:,:,sl])
                    nawm_slice = nawm[:,:,sl]
                    #plt.figure(figsize=(10, 5))
                    #plt.subplot(1, 2, 1)
                    #plt.imshow(nawm_slice, cmap='gray')
                    #plt.subplot(1, 2, 2)
                    #plt.imshow(current_slice, cmap='gray')
                    #plt.imshow(lm_slice, cmap='Reds', alpha=0.6)
                    #plt.show()
                    nawm_slice = nawm_slice[nawm_slice>0]
                    assert(nawm_slice.any() > 0)
                    mean, std = np.mean(nawm_slice), np.std(nawm_slice)
                    if np.isnan(std):
                        print(nawm_slice)
                    current_slice[np.greater(lm_slice,0)] = np.random.normal(loc=mean, scale=std/2)
                    #if np.max(current_slice) <150:
                    #    print("Slice {}".format(sl))
                    #    print("NAWM mean {}".format(np.mean(nawm_slice)))
                    #    print("lesion mean {}".format(np.mean(lm_slice)))
                    #    print("current Slice mean {}".format(np.mean(current_slice)))
                    scan[:,:,sl] = current_slice
                    
            """print(np.mean(lm))
            print(np.max(scan))
            plt.subplot(1, 2, 2)
            plt.imshow(scan[:,:,96], cmap='gray')
            plt.title("After")
            plt.show()
            
            plt.imshow(lm[:,:,96], cmap='Reds')
            plt.show()
            """
            #plt.figure()
            #plt.subplot(1, 2, 1)
            #plt.imshow(scan[:,:,96], cmap='gray')
            #plt.imshow(lm[:,:,96], cmap='Reds', alpha=0.4)
            #plt.subplot(1, 2, 2)
            #plt.imshow(wm[:,:,96], cmap='gray')
            #plt.show()
            output_path = path.replace(".nii.gz", "_lesions_filled_valverde.nii.gz")
            out_nifti = nib.Nifti1Image(scan, nifti.affine, header=nifti.header)
            nib.save(out_nifti, output_path)
            
        except(FileNotFoundError):
            print("File not Found, skipping: {}".format(path))


In [43]:
fill_lesions(df)

/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_027/VIMS_MS_027_1/BET_VIMS_MS_027_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_043/VIMS_MS_043_1/BET_VIMS_MS_043_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_141/VIMS_MS_141_1/BET_VIMS_MS_141_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_137/VIMS_MS_137_1/BET_VIMS_MS_137_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/VIMS_HC_022/VIMS_HC_022_1/BET_VIMS_HC_022_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/VIMS_HC_114/VIMS_HC_114_1/BET_VIMS_HC_114_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/RS_GK_021/RS_GK_021_1/BET_RS_GK_021_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_088/VIMS_MS_088_1/BET_VIMS_MS_088_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/VIMS_HC_045/VIMS_HC_045_1/BET_VIMS_HC_045_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_071/VIMS_MS_071_1/BET_VIMS_MS_071_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/RS_GK_012/RS_GK_012_1/BET_RS_GK_012_1_FL

/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_049/VIMS_MS_049_1/BET_VIMS_MS_049_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/RS_GK_005/RS_GK_005_1/BET_RS_GK_005_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_072/VIMS_MS_072_1/BET_VIMS_MS_072_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/VIMS_HC_097/VIMS_HC_097_1/BET_VIMS_HC_097_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_001/VIMS_MS_001_1/BET_VIMS_MS_001_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/RS_GK_023/RS_GK_023_1/BET_RS_GK_023_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_041/VIMS_MS_041_1/BET_VIMS_MS_041_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/RS_GK_006/RS_GK_006_1/BET_RS_GK_006_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/VIMS_HC_033/VIMS_HC_033_1/BET_VIMS_HC_033_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/02_HC/VIMS_HC_060/VIMS_HC_060_1/BET_VIMS_HC_060_1_FLAIR.nii.gz
/analysis/share/Ritter/MS/CIS/03_MS/VIMS_MS_143/VIMS_MS_143_1/BET_VIMS_MS_143_1_FLAIR.ni